In [46]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, TFAutoModelForCausalLM, AutoModelForQuestionAnswering, Wav2Vec2ForCTC
from transformers import pipeline
from pprint import pprint

# 1. Read the documentation of Language modelling in the Transformers library.
# 2. Download three Polish models from the Huggingface repository.

In [31]:
# tokenizer_kleczek_uncased = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
# model_kleczek_uncased = AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
tokenizer_kleczek_cased = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")
model_kleczek_cased = AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
# tokenizer_henryk_cased = AutoTokenizer.from_pretrained("henryk/bert-base-multilingual-cased-finetuned-polish-squad2")
# model_henryk_cased = AutoModelForQuestionAnswering.from_pretrained("henryk/bert-base-multilingual-cased-finetuned-polish-squad2")
# tokenizer_jonatasgrosman = AutoTokenizer.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-polish")
# model_jonatasgrosman = Wav2Vec2ForCTC.from_pretrained("jonatasgrosman/wav2vec2-large-xlsr-53-polish") 
tokenizer_geotrend_distilbert = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-pl-cased")
model_geotrend_distilbert = AutoModelForMaskedLM.from_pretrained("Geotrend/distilbert-base-pl-cased")
tokenizer_geotrend_bert = AutoTokenizer.from_pretrained("Geotrend/bert-base-pl-cased")
model_geotrend_bert = AutoModelForMaskedLM.from_pretrained("Geotrend/bert-base-pl-cased")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100%|██████████| 49.0/49.0 [00:00<00:00, 24.5kB/s]
Downloading: 100%|██████████| 557/557 [00:00<00:00, 558kB/s]
Downloading: 100%|██████████| 138k/138k [00:00<00:00, 434kB/s] 
Downloading: 100%|██████████| 232M/232M [00:21<00:00, 11.1MB/s]
Downloading: 100%|██████████| 49.0/49.0 [00:00<00:00, 24.5kB/s]
Downloading: 100%|██████████| 752

# 3. Produce the predictions for the following sentences (use each model and check 5 predictions):

- (M) Warszawa to największe [MASK].
- (D) Te zabawki należą do [MASK].
- (C) Policjant przygląda się [MASK].
- (B) Na środku skrzyżowania widać [MASK].
- (N) Właściciel samochodu widział złodzieja z [MASK].
- (Ms) Prezydent z premierem rozmawiali wczoraj o [MASK].
- (W) Witaj drogi [MASK].


In [22]:
texts_1 = {
    'M': "Warszawa to największe [MASK].",
    'D': "Te zabawki należą do [MASK].",
    'C': "Policjant przygląda się [MASK].",
    'B': "Na środku skrzyżowania widać [MASK].",
    'N': "Właściciel samochodu widział złodzieja z [MASK].",
    'Ms': "Prezydent z premierem rozmawiali wczoraj o [MASK].",
    'W': "Witaj drogi [MASK].",
}

In [ ]:
nlp_k_c = pipeline('fill-mask', model=model_kleczek_cased, tokenizer=tokenizer_kleczek_cased)
nlp_g_d = pipeline('fill-mask', model=model_geotrend_distilbert, tokenizer=tokenizer_geotrend_distilbert)
nlp_g_b = pipeline('fill-mask', model=model_geotrend_bert, tokenizer=tokenizer_geotrend_bert)

In [52]:
task_1_results = {}
for id, sentence in texts_1.items():
    if id not in task_1_results:
        task_1_results[id] = {}
    part_first, part_second = sentence.split("[MASK]")
    task_1_results[id]["sentence"] = sentence
    task_1_results[id]["k_c"] = [word['token_str'] for word in nlp_k_c(f"{part_first}{nlp_k_c.tokenizer.mask_token}{part_second}")]
    task_1_results[id]["g_d"] = [word['token_str'] for word in nlp_g_d(f"{part_first}{nlp_g_d.tokenizer.mask_token}{part_second}")]
    task_1_results[id]["g_b"] = [word['token_str'] for word in nlp_g_b(f"{part_first}{nlp_g_b.tokenizer.mask_token}{part_second}")]
pprint(task_1_results, sort_dicts=False)


{'M': {'sentence': 'Warszawa to największe [MASK].',
       'k_c': ['miasto', 'województwo', 'lotnisko', 'miasteczko', 'państwo'],
       'g_d': ['miasto', 'miasta', 'Miasto', 'centrum', 'dzielnicy'],
       'g_b': ['miasto', 'miasta', 'woj', 'Warszawa', 'miast']},
 'D': {'sentence': 'Te zabawki należą do [MASK].',
       'k_c': ['ciebie', 'mnie', 'nas', 'pana', 'niego'],
       'g_d': ['klasyfikacji', 'gry', 'rodziny', 'grupy', 'zespołu'],
       'g_b': ['tzw', 'pt', 'pl', 'odc', 'ok']},
 'C': {'sentence': 'Policjant przygląda się [MASK].',
       'k_c': ['temu', 'sprawie', 'im', 'wszystkiemu', 'panu'],
       'g_d': ['przeciwko', 'LGBT', 'walki', 'nie', 'ludzi'],
       'g_b': ['go', 'ok', 'się', 'pt', 'nie']},
 'B': {'sentence': 'Na środku skrzyżowania widać [MASK].',
       'k_c': ['rzekę', 'ulicę', 'drzewa', 'drogę', 'las'],
       'g_d': ['wody', '##ały', 'miejsca', 'brak', '##ły'],
       'g_b': ['św', 'ok', 'obraz', 'rok', 'kościół']},
 'N': {'sentence': 'Właściciel samochodu w

# 4. Check the model predictions for the following sentences (using each model):

- Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].
- Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].


In [53]:
texts_2 = {
    'T1': "Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].",
    'T2': "Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].",
}

In [54]:
task_2_results = {}
for id, sentence in texts_2.items():
    if id not in task_2_results:
        task_2_results[id] = {}
    part_first, part_second = sentence.split("[MASK]")
    task_2_results[id]["sentence"] = sentence
    task_2_results[id]["k_c"] = [word['token_str'] for word in nlp_k_c(f"{part_first}{nlp_k_c.tokenizer.mask_token}{part_second}")]
    task_2_results[id]["g_d"] = [word['token_str'] for word in nlp_g_d(f"{part_first}{nlp_g_d.tokenizer.mask_token}{part_second}")]
    task_2_results[id]["g_b"] = [word['token_str'] for word in nlp_g_b(f"{part_first}{nlp_g_b.tokenizer.mask_token}{part_second}")]
pprint(task_2_results, sort_dicts=False)

{'T1': {'sentence': 'Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym '
                    'się nie [MASK].',
        'k_c': ['zgodził', 'bał', 'dowiedział', 'pojawił', 'zabił'],
        'g_d': ['było', 'stanie', 'udało', 'tylko', 'odbył'],
        'g_b': ['##wiedział', 'było', 'stało', '##dził', 'stanie']},
 'T2': {'sentence': 'Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to '
                    'bym się nie [MASK].',
        'k_c': ['zgodziła', 'bała', 'dowiedziała', 'pojawiła', 'zabiła'],
        'g_d': ['było', 'udało', 'stanie', 'tylko', 'ma'],
        'g_b': ['było', '##wia', 'stało', '##śli', '##wiedział']}}


# 5. Check the model predictions for the following sentences:

- [MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
- W wakacje odwiedziłem [MASK], który jest stolicą Islandii.
- Informatyka na [MASK] należy do najlepszych kierunków w Polsce.


In [55]:
texts_3 = {
    'T1': "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.",
    'T2': "W wakacje odwiedziłem [MASK], który jest stolicą Islandii.",
    'T3': "Informatyka na [MASK] należy do najlepszych kierunków w Polsce.",
}

In [56]:
task_3_results = {}
for id, sentence in texts_3.items():
    if id not in task_3_results:
        task_3_results[id] = {}
    part_first, part_second = sentence.split("[MASK]")
    task_3_results[id]["sentence"] = sentence
    task_3_results[id]["k_c"] = [word['token_str'] for word in nlp_k_c(f"{part_first}{nlp_k_c.tokenizer.mask_token}{part_second}")]
    task_3_results[id]["g_d"] = [word['token_str'] for word in nlp_g_d(f"{part_first}{nlp_g_d.tokenizer.mask_token}{part_second}")]
    task_3_results[id]["g_b"] = [word['token_str'] for word in nlp_g_b(f"{part_first}{nlp_g_b.tokenizer.mask_token}{part_second}")]
pprint(task_3_results, sort_dicts=False)

{'T1': {'sentence': '[MASK] wrze w temperaturze 100 stopni, a zamarza w '
                    'temperaturze 0 stopni Celsjusza.',
        'k_c': ['Woda', 'Mięso', 'Słońce', 'Nie', 'Ziemia'],
        'g_d': ['Na', 'We', 'Od', 'Maja', 'Zmarł'],
        'g_b': ['Jego', 'Za', 'Po', 'Nie', 'Ich']},
 'T2': {'sentence': 'W wakacje odwiedziłem [MASK], który jest stolicą '
                    'Islandii.',
        'k_c': ['kraj', 'Cypr', 'Meksyk', 'Gibraltar', 'Wellington'],
        'g_d': ['kraju', 'pochodzi', 'flag', 'wody', 'referendum'],
        'g_b': ['Island', 'Reykjavík', 'Porto', 'miasto', 'Port']},
 'T3': {'sentence': 'Informatyka na [MASK] należy do najlepszych kierunków w '
                    'Polsce.',
        'k_c': ['wsi', 'świecie', 'żywo', 'pewno', 'odległość'],
        'g_d': ['stacji', 'Uniwersytecie', 'Ziemi', 'terenie', 'ulicy'],
        'g_b': ['uczelni', 'rynku', 'internet', 'świecie', 'terenie']}}


# 6. Answer the following questions:

1. Which of the models produced the best results?
    Najlepiej wypadł bert od Kłeczka. Generalnie dawał on najlepsze wyniki. 

2. Was any of the models able to capture Polish grammar?
    Bert od Kłeczka w większosci przypadków robił to poprawnie. Czasami jeszcze distilbert od Geotrend.

3. Was any of the models able to capture long-distant relationships between the words?
    Raczej żaden w pełni, każdy po trochę.

4. Was any of the models able to capture world knowledge?
    Tylko bertowi od Kłeczka się udało.

5. What are the most striking errors made by the models?
    Większość generalnie próbuje generowac krótkie słowa, raczej spójniki i przyimki. Jak sa zdania złożone to nie rozumieją kontekstu.

